# Udacity ML Azure, Project 1: Optimizing a Machine Learning Pipeline

Here 2 approaches for optimizing machine learning pipelines are trialled and the results compared.
1. Hyperdrive: Build and optimize an Azure ML pipeline using the Python SDK and a provided Scikit-learn model.
2. Azure AutoML.

Please also see:  https://knowledge.udacity.com/questions/504246

## Initialize Workspace

In [1]:
from azureml.core import Workspace, Experiment

# Note: this requires the workspace name to be known at the point of editing this notebook:
#ws = Workspace.get(name="udacity-project")

# An alternative approach is to use the "ws = Workspace.from_config()" method.
# This fetches the workspace details from the config file created by Azure when the session starts.
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-140280
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-140280


# Hyperdrive
## 1. Create a compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException  # Use when verifying whether cluster already exists

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Useful refs for verifying whether cluster exists and provisioning a compute cluster:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.ipynb

# Set the compute cluster name
cluster_name = "azml-cluster"

# Verify that cluster does not exist already
# If cluster already exists use it, otherwise initialize the cluster (exception thrown if cluster does not exist)
try:
    azml_cluster = ComputeTarget(ws, cluster_name)  # exists
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Set up the compute cluster config
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
    # Create the compute cluster
    azml_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

# Wait for the current provisioning operation to finish on the cluster.
azml_cluster.wait_for_completion(show_output = True)

# Display status information about the AmlCompute target:
# Fetch AmlComputeStatus object using ComputeTarget's get_status() method
# AmlComputeStatus: Represents detailed status information about an AmlCompute target.
# Convert the AmlComputeStatus object into a JSON serialized dictionary using serialize().
# Ref: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcomputestatus
print(azml_cluster.get_status().serialize())


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-12T14:48:12.667000+00:00', 'errors': None, 'creationTime': '2021-03-12T14:48:08.905156+00:00', 'modifiedTime': '2021-03-12T14:48:25.646858+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## 2. Set up the Hyperdrive configuration

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling
param_dict = {
    "--C" : uniform(0.001, 2.0),
    "--max_iter" : choice(10,25,50,100,150,200,250)
}
ps = RandomParameterSampling(param_dict)

# Specify a Policy
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn
est = SKLearn(source_directory=".",
              compute_target=azml_cluster, 
              vm_size="Standard_D2_V2", 
              vm_priority="dedicated", 
              entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy", 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     policy=policy, 
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## 3. Submit Hyperdrive run

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hd_run = exp.submit(hyperdrive_config)
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## 4. Get the best Hyperdrive run and save the model

In [6]:
import joblib
# Get your best run and save the model from that run.
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print("best_run_metrics:",best_run_metrics)

best_run_metrics: {'Regularization Strength:': 1.6807881339560455, 'Max iterations:': 25, 'Accuracy': 0.9107044325915027}


In [8]:
# Implementation for saving the model guided by information here:
# https://knowledge.udacity.com/questions/424266
# https://www.kaggle.com/pankaj1234/azure-machine-learning-model-training 
# https://towardsdatascience.com/azure-machine-learning-service-train-a-model-df72c6b5dc
print("Filenames:",best_run.get_file_names()) # print the filenames
print("Run Details:",best_run.get_details()['runDefinition']['arguments']) # print the run details

Filenames: ['azureml-logs/55_azureml-execution-tvmps_8b7bb7d43ee67b8a9ad25f1b41545ffa8886017ff5dd93d3d5458687683e8cbc_d.txt', 'azureml-logs/65_job_prep-tvmps_8b7bb7d43ee67b8a9ad25f1b41545ffa8886017ff5dd93d3d5458687683e8cbc_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_8b7bb7d43ee67b8a9ad25f1b41545ffa8886017ff5dd93d3d5458687683e8cbc_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/103_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/my_model.joblib']
Run Details: ['--C', '1.6807881339560455', '--max_iter', '25']


In [10]:
print("Best Run ID:",best_run.id)
print("Accuracy:",best_run_metrics["Accuracy"])

Best Run ID: HD_17aefb7a-a94c-4e0b-8507-ec4a2938f198_0
Accuracy: 0.9107044325915027


In [11]:
# Display details of the best run
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_17aefb7a-a94c-4e0b-8507-ec4a2938f198_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
# Download the model to the local file system from the best run using the run object’s download_file() method.
# Note: The model will be the last file in the list hence the -1 index can be used to reference it.
os.makedirs("outputs", exist_ok=True)  # Ensure that there is a local outputs folder
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/my_best_hd_model.joblib')  # Download

In [13]:
# Verify that best model has been retrieved
joblib.load('./outputs/my_best_hd_model.joblib')

The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.


LogisticRegression(C=1.6807881339560455, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=25, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
# Register the model
best_hyperdrive_model = best_run.register_model(
    model_name="best_hyperdrive_model",
    model_path="./outputs/my_model.joblib",  # Cloud path
    tags=best_run.get_metrics()
)

# -----------------------------------------------------------------------------------------------------------
# AutoML

Note: At the time this project was completed there was a version mismatch between the default xgboost version available onthe compute-instance and that used during AutoML training. As a consequence it was necessary to reinstall xgboost using a version which matched that used by AutoML. See this knowledge thread for more details: https://knowledge.udacity.com/questions/504246

## 0. Checking current status of installed packages

In [3]:
# Relevant info in this Udacity Knowledge thread: 
# https://knowledge.udacity.com/questions/504246
# After completion of the AutoML run, when attempting to retrieve the best run and the best model the 
# following error occurs:
# AttributeError: /anaconda/envs/azureml_py36/lib/libxgboost.so: undefined symbol: XGBoosterUnserializeFromBuffer
# It has been established that there is a version mismatch between the xgboost version install via pip on the 
# compute-instance (where this notebook runs) and the py-xgboost version (conda) used during AutoML on the 
# compute-cluster.

# Resolution of this issue will be attempted by: 
# 1. Establishing which package versions are present in this notebook.
# 2. Installing versions which match those used during the AutoML run.

# Display installed xgboost version
import xgboost
print(xgboost.__version__)

0.90


In [4]:
# Display pip list
!pip list

Package                               Version
------------------------------------- -----------------
absl-py                               0.11.0
adal                                  1.2.6
aiohttp                               3.7.3
aiohttp-cors                          0.7.0
aioredis                              1.3.1
alembic                               1.4.1
ansiwrap                              0.8.4
antlr4-python3-runtime                4.7.2
applicationinsights                   0.11.9
argcomplete                           1.12.2
argon2-cffi                           20.1.0
astor                                 0.8.1
astroid                               2.4.2
astunparse                            1.6.3
async-timeout                         3.0.1
atari-py                              0.2.6
attrs                                 20.3.0
autokeras                             1.0.12
autopep8                              1.5.5
azure-appconfiguration                1.1.1
azure-batch 

In [5]:
# Display conda list
!conda list

# packages in environment at /anaconda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
applicationinsights       0.11.9                   pypi_0    pypi
brotlipy                  0.7.0           py38h27cfd23_1003  
ca-certificates           2020.10.14                    0    anaconda
certifi                   2020.6.20                py38_0    anaconda
cffi                      1.14.3           py38h261ae71_2  
chardet                   3.0.4           py38h06a4308_1003  
conda                     4.9.2            py38h06a4308_0  
conda-package-handling    1.7.2            py38h03888b9_0  
cryptography              3.2.1            py38h3c74f83_1  
idna                      2.10                       py_0  
ld_impl_linux-64          2.33.1               h53a641e_7  
libedit                   3.1.20191231         h14c3975_1  
libffi                    3.3                  he6710b0_2  
libgcc-ng            

## 1. Read in the dataset

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

raw_data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(raw_data_url)

## 2. Prepare the data

Clean the data and divide into training and test sets.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from train import clean_data

# Use the clean_data function to clean the data (as per project instructions).
x, y = clean_data(ds)

# Split into train and test set (as per project instructions)
# Note: nothing has been specified in the project instructions or the project rubric with respect to 
# the use of the test set...
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=8, stratify=y)

# Training data as DataFrame (joining input and target feature again into one DataFrame)
df_train = pd.concat([x_train,y_train], axis=1)




## 3. Set the parameters of the AutoMLConfig

In [8]:
from azureml.train.automl import AutoMLConfig

# Get the default datastore
datastore = ws.get_default_datastore()

# Convert the DataFrame into a TabularDataset
train_data = TabularDatasetFactory.register_pandas_dataframe(df_train, datastore, "train_data")

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=azml_cluster)

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/0827c98e-1832-4c56-9fe3-91f85bd60f8c/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## 4. Submit the AutoML run

In [9]:
from azureml.widgets import RunDetails #imported again here as AutoML rerun in second attempt
# Submit your automl run
experiment = Experiment(workspace=ws, name="automl_expt")
automl_run = experiment.submit(config=automl_config, show_output=True)
# View progress
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on azml-cluster with default configuration
Running on remote compute: azml-cluster
Parent Run ID: AutoML_dbd7cde5-0c23-491b-9938-aad75461e83e

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a f

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:01:01       0.9136    0.9136
         1   MaxAbsScaler XGBoostClassifier                 0:01:04       0.9143    0.9143
         2   MaxAbsScaler RandomForest                      0:00:52       0.8897    0.9143
         3   MaxAbsScaler RandomForest                      0:00:52       0.8879    0.9143
         4   MaxAbsScaler RandomForest                      0:00:52       0.8176    0.9143
         5   MaxAbsScaler RandomForest                      0:00:56       0.7743    0.9143
         6   SparseNormalizer XGBoostClassifier             0:01:19       0.9117    0.9143
         7   MaxAbsS

{'runId': 'AutoML_dbd7cde5-0c23-491b-9938-aad75461e83e',
 'target': 'azml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-12T14:51:25.06342Z',
 'endTimeUtc': '2021-03-12T15:38:45.20326Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'azml-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_expt","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-140280","workspace_name":"quick-starts-ws-140280","region":"southcentralus","compute_target":"azml-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"fe

## 5. Retrieve and save the best model

Note: as a prestep it is currently necessary to reinstall xgboost using a version which matches that used by AutoML.
See this knowledge thread for more details: https://knowledge.udacity.com/questions/504246

In [ ]:
# Uninstall the current version of xgboost
!pip uninstall xgboost

In [ ]:
# Install the version of py-xgboost which matches that used by AutoML (see the environment raw json retrieved 
# from one of the runs).
!conda install -c conda-forge py-xgboost==0.90

In [ ]:
# import xgboost again and check the version:
import importlib
importlib.reload(xgboost)
print(xgboost.__version__)

In [10]:
# Retrieve and save your best automl model.
best_automl_run, best_automl_model = automl_run.get_output()

Package:azureml-automl-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-core, training version:1.23.0, current version:1.22.0
Package:azureml-dataprep, training version:2.10.1, current version:2.9.1
Package:azureml-dataprep-native, training version:30.0.0, current version:29.0.0
Package:azureml-dataprep-rslex, training version:1.8.0, current version:1.7.0
Package:azureml-dataset-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-defaults, training version:1.23.0, current version:1.22.0
Package:azureml-interpret, training version:1.23.0, current version:1.22.0
Package:azureml-mlflow, training version:1.23.0, current version:1.22.0
Package:azureml-pipeline-core, training version:1.23.0, current version:1.22.0
Package:azureml-telemetry, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-client, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-runtime, training version:1.23.0, current versio

In [17]:
print(best_automl_run)

Run(Experiment: automl_expt,
Id: AutoML_dbd7cde5-0c23-491b-9938-aad75461e83e_21,
Type: azureml.scriptrun,
Status: Completed)


In [18]:
print(best_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_samples=None,
                                                                                                    min_impurity_decrease=0.0,
                                                                                                    min_impurity_split=None,
                      

In [12]:
print("Filenames (best AutoML run):",best_automl_run.get_file_names()) # print the filenames
print("Tags (best AutoML run):",best_automl_run.get_tags())

Filenames (best AutoML run): ['accuracy_table', 'automl_driver.py', 'azureml-logs/55_azureml-execution-tvmps_891ebd7f26a22a52088674c4977be484eba104be37f2043dd1e2eb47476c4438_d.txt', 'azureml-logs/65_job_prep-tvmps_891ebd7f26a22a52088674c4977be484eba104be37f2043dd1e2eb47476c4438_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_891ebd7f26a22a52088674c4977be484eba104be37f2043dd1e2eb47476c4438_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'confusion_matrix', 'logs/azureml/113_azureml.log', 'logs/azureml/azureml_automl.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/conda_env_v_1_0_0.yml', 'outputs/env_dependencies.json', 'outputs/internal_cross_validated_models.pkl', 'outputs/model.pkl', 'outputs/pipeline_graph.json', 'outputs/scoring_file_v_1_0_0.py']
Tags (best AutoML run): {'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","P

In [ ]:
# Download the model TODO Delete
#best_automl_run.download_file(best_automl_run.get_file_names()[-1], output_file_path='./outputs/')

In [23]:
best_automl_run.get_metrics()

{'precision_score_macro': 0.7947059920235364,
 'f1_score_weighted': 0.9154236405025417,
 'f1_score_micro': 0.9169684519999342,
 'balanced_accuracy': 0.7740958611942405,
 'weighted_accuracy': 0.9525135199650201,
 'average_precision_score_weighted': 0.9544669097596421,
 'norm_macro_recall': 0.5481917223884809,
 'log_loss': 0.24419932540433914,
 'f1_score_macro': 0.7837069273962338,
 'AUC_macro': 0.9456624915400583,
 'recall_score_macro': 0.7740958611942405,
 'recall_score_weighted': 0.9169684519999344,
 'precision_score_weighted': 0.914250497149844,
 'average_precision_score_micro': 0.9807100986201638,
 'average_precision_score_macro': 0.8215191553046806,
 'AUC_micro': 0.9799638755726303,
 'accuracy': 0.9169684519999344,
 'matthews_correlation': 0.5683204041707439,
 'AUC_weighted': 0.9456624915400583,
 'recall_score_micro': 0.9169684519999344,
 'precision_score_micro': 0.9169684519999344,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_dbd7cde5-0c23-491b-9938-aad75461e83

In [20]:
import joblib
# Save with joblib
joblib.dump(value=best_automl_model, filename='outputs/automl_best_model.joblib')

['outputs/automl_best_model.joblib']

In [24]:
registered_best_automl_model = best_automl_run.register_model(
    model_name = 'automl_best_model', 
    model_path = './outputs/',
    tags=best_automl_run.get_metrics())

## 6. Additional step - test the best AutoML model 

Use the test set to make predictions using the trained model and display the results.

In [30]:
x_test.columns


Index(['age', 'marital', 'default', 'housing', 'loan', 'month', 'day_of_week',
       'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'contact_cellular', 'contact_telephone', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown'],
      dtype='object')

In [35]:
df_train.columns

Index(['age', 'marital', 'default', 'housing', 'loan', 'month', 'day_of_week',
       'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'contact_cellular', 'contact_telephone', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'y'],
      dtype='object')

In [39]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models
y_predict = best_automl_model.predict(x_train)

DataException: DataException:
	Message: Expected column(s) emp.var.rate not found in fitted data.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Expected column(s) emp.var.rate not found in fitted data.",
        "target": "X",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "MissingColumnsInData"
            }
        },
        "reference_code": "17049f70-3bbe-4060-a63f-f06590e784e5"
    }
}

In [ ]:
# Prepare the target values for use in scoring
y_true = y_test.values.flatten().tolist()

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
tn, fp, fn, tp = confusion_matrix(y_true, y_predict).ravel()
print("True Negative Count:", tn)
print("False Positive Count:", fp)
print("False Negative Count:", fn)
print("True Positive Count:", tp)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
target_names = ['NO', 'YES']
print(classification_report(y_true, y_pred, target_names=target_names))

## -------------------------------------------------------------------------------------------------------------------------------
## Clean up deployed resources.

In [11]:
# Delete the compute cluster 
azml_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

